# With ChatModel

In [3]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser

In [4]:
def save_result(output, prompt_type, model, init, best_practice, output_parser):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice}
    result['output_parser'] = output_parser 
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [5]:
def StringToJSON(string):
    result = string

    try:
        if ((string[0] != "{") and (string[0] != "[")):
            string = "{"+string+"}"

        result = json.loads(string)

    except Exception as e:
        print("JSON formatting went wrong. The result returns as a string.")
        
    finally:
        return result

In [6]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

In [7]:
with open('./data/json/last_row/last_row.json') as f:
    json_sample = json.load(f)

## Custom Parsers

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field, validator
# from langchain.output_parsers import PydanticOutputParser

# class ListItem(BaseModel):
#     """Item of a list"""

#     id: int = Field(description="The item identifier")
#     content: str = Field(description="The item content")

#     @validator("id")
#     def validate_id(cls, field):
#         if not field:
#             raise ValueError("Item ID cannot be empty!")
#         return field
    
#     @validator("content")
#     def validate_content(cls, field):
#         if not field:
#             raise ValueError("Item content cannot be empty!")
#         return field
    
# parser = PydanticOutputParser(pydantic_object=ListItem)
# ...
# bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format(format_instructions=parser.get_format_instructions())

## Summary Length

In [13]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'summaryLengthPrompt_V2.2.0.json'
bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min=39, max=70)

In [14]:
chain = init_prompt | model | StrOutputParser()

In [15]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

'"SourceTree crashes upon opening, despite multiple reinstallations"'

In [16]:
# should be between 39 and 70
len(output)

67

In [17]:
save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')


In [31]:
# # maybe usefull for later
# def run(model_name, init_prompt_name, sumLen_prompt_name):
#     model = ChatOpenAI(
#         model=model_name,
#         api_key=open('api.txt', 'r').read(),
#         temperature=0
#     )
#     with open('./data/json/last_row.json') as f:
#         json_sample = json.load(f)

#     init_prompt = load_prompt('prompts/init/'+init_prompt_name)
#     sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)
    
#     chain = init_prompt | model | StrOutputParser()

#     output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})

#     save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)  

# run('gpt-4', 'initPrompt_V1.0.0.json', 'summaryLengthPrompt_V1.0.1.json')  

## Description Length

In [39]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionLengthPrompt_V1.0.2.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format(min= 29, max= 111)

chain = init_prompt | model | StrOutputParser()

In [40]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

"'Source tree crashes every time I open it. I have uninstalled and reinstalled it multiple times but the issue persists. This issue is occurring in version 3.4.4. I need assistance in resolving this issue as it is hindering my work.'"

In [41]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Field Update

In [110]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.12.2.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

In [111]:
chain = init_prompt | model | StrOutputParser()

In [112]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})

In [113]:
output_json = StringToJSON(output)

In [114]:
save_result(output_json, 'update', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Toxic Speech Detection

In [115]:
with open('./data/json/last_row/last_row_toxic.json') as f:
    toxic_sample = json.load(f)

In [116]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.2.4.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

chain = init_prompt | model | StrOutputParser()

In [117]:
output = chain.invoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":toxic_sample})

In [118]:
output_json = StringToJSON(output)

In [119]:
save_result(output_json, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Bug Report Structure

In [120]:
bugReportStrucutre_prompt_name = 'bugReportStructurePrompt_V1.0.1.json'
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+bugReportStrucutre_prompt_name)

In [121]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [122]:
output = chain.invoke({"role":"Software Engineer", "bug_Report":json_sample})

In [123]:
save_result(output, 'bugReportStructure', model_name, '-', bugReportStrucutre_prompt_name, 'StrOutputParser')

## Internationalization

In [145]:
with open('./data/json/last_row/last_row_german.json') as f:
    german_sample = json.load(f)

In [146]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'internationalizationPrompt_V1.2.0.json'
bestPractice_prompt = load_prompt('prompts/internationalization/'+bestPractice_prompt_name).format()

In [147]:
chain = init_prompt | model | StrOutputParser()

In [148]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":german_sample})

In [149]:
output_json = StringToJSON(output)

{"Summary": "Sourcetree crashes",
"Description": "The Sourcetree crashes every time I open it, I have uninstalled and installed several times"}


In [150]:
save_result(output_json, 'internationalization', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')